In [1]:
# ── Cell 1: Initialization & Imports ─────────────────────────────────
# Enable auto-reload so changes to derivatives.py are picked up without restarting the kernel.
%load_ext autoreload
%autoreload 2

# Standard libraries
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

# Import our option classes (with full Greeks) from derivatives.py
from derivatives import EuropeanCall, AmericanPut, BarrierOption, BasketCall

print("✅ Loaded derivatives.py and core libraries.")


✅ Loaded derivatives.py and core libraries.


In [2]:
# ── Cell 2: Spot Prices & Discount Function ─────────────────────────
# Fetch ASX closing prices for our tickers on the trade date
trade_date = "2025-05-16"
next_day   = (pd.to_datetime(trade_date) + pd.Timedelta(days=1)).strftime("%Y-%m-%d")

tickers = ["BHP.AX", "CBA.AX", "WES.AX", "CSL.AX", "WDS.AX", "MQG.AX"]
df_spot  = yf.download(tickers, start=trade_date, end=next_day, progress=False)["Close"]
S0       = df_spot.loc[trade_date].to_dict()  # spot prices dict

# Define a flat-curve discount function (5% p.a.) as placeholder
r        = 0.05
discount = lambda t: np.exp(-r * t)

print("Spots (S0):", S0)
print(f"Using flat risk-free rate r = {r:.2%}")


YF.download() has changed argument auto_adjust default to True
Spots (S0): {'BHP.AX': 39.720001220703125, 'CBA.AX': 169.66000366210938, 'CSL.AX': 241.82000732421875, 'MQG.AX': 207.35000610351562, 'WDS.AX': 21.920000076293945, 'WES.AX': 82.55999755859375}
Using flat risk-free rate r = 5.00%


In [3]:
# ── Cell 3: Realised Volatility Calculation ─────────────────────────
# Download 1 year of historical closes to compute realised vols
hist = yf.download(tickers, end=trade_date, period="1y", progress=False)["Close"]
rets = hist.pct_change().dropna()                # daily returns
vol  = (rets.std() * np.sqrt(252)).to_dict()     # annualised vol σ√252

print("Realised vols (annualised):")
for tkr, σ in vol.items():
    print(f"  {tkr:7s}: {σ:.2%}")


Realised vols (annualised):
  BHP.AX : 22.78%
  CBA.AX : 21.48%
  CSL.AX : 19.12%
  MQG.AX : 24.73%
  WDS.AX : 28.90%
  WES.AX : 20.72%


In [4]:
# ── Cell 4: Instantiate Trades & Compute Greeks (with manual basket delta) ──

import numpy as np
import pandas as pd

# Helper to compute year fraction between two dates
def year_frac(start, end):
    return (pd.to_datetime(end) - pd.to_datetime(start)).days / 365

# 1) BHP European Call
T1  = year_frac(trade_date, "2027-09-15")
bhp = EuropeanCall(
    S0       = S0["BHP.AX"],
    K        = 0.98 * S0["BHP.AX"],
    T        = T1,
    discount = discount,
    sigma    = vol["BHP.AX"]
)

# 2) CBA American Put
T2  = year_frac(trade_date, "2026-05-15")
cba = AmericanPut(
    S0       = S0["CBA.AX"],
    K        = 170.0,
    T        = T2,
    discount = discount,
    sigma    = vol["CBA.AX"]
)

# 3) WES Up-and-In Barrier Call
wes = BarrierOption(
    S0       = S0["WES.AX"],
    K        = 80.0,
    T        = T1,
    discount = discount,
    sigma    = vol["WES.AX"],
    barrier  = 100.0
)

# 4) Basket Call on [BHP, CSL, WDS, MQG]
T3     = year_frac(trade_date, "2025-07-17")
tick4  = ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]
weights = np.array([0.10, 0.35, 0.15, 0.40])
basket = BasketCall(
    S0_list    = np.array([S0[t] for t in tick4]),
    weights    = weights,
    K          = 175.0,
    T          = T3,
    discount   = discount,
    sigma_list = np.array([vol[t] for t in tick4]),
    corr       = np.identity(4),
    paths      = 50000
)

# Compute price, delta, theta, and vega for the first three trades
results = []
for name, obj in [
    ("BHP Euro Call", bhp),
    ("CBA Am Put",    cba),
    ("WES Barrier",   wes)
]:
    p   = obj.price()
    d   = obj.delta()
    th  = obj.theta()
    vg  = getattr(obj, "vega", lambda: np.nan)()
    results.append((name, p, d, th, vg))

# ── Manual basket delta calculation ──────────────────────────────────
#  Compute partial ∂P/∂S_i by bumping each S0[i] by +1
base_price = basket.price()
eps        = 1.0
partials   = []
for i in range(len(weights)):
    bumped_S0 = basket.S0_list.copy()
    bumped_S0[i] += eps
    bumped = BasketCall(
        S0_list    = bumped_S0,
        weights    = weights,
        K          = basket.K,
        T          = basket.T,
        discount   = basket.discount,
        sigma_list = basket.sigma_list,
        corr       = basket.corr,
        paths      = basket.paths
    )
    p_bumped = bumped.price()
    partials.append((p_bumped - base_price) / eps)

delta_basket = np.dot(weights, partials)
theta_basket = basket.theta()
vega_basket  = basket.vega()

# Append basket results
results.append(("Basket Call", base_price, delta_basket, theta_basket, vega_basket))

# (Optional) inspect partials
print("Basket partial deltas:", np.round(partials, 4))
print("Basket portfolio delta:", round(delta_basket, 4))

results


Basket partial deltas: [0.1212 0.2415 0.1756 0.3296]
Basket portfolio delta: 0.2548


[('BHP Euro Call',
  np.float64(8.064656731137958),
  np.float64(0.7147894855030756),
  np.float64(-2.0223237365429725),
  nan),
 ('CBA Am Put',
  np.float64(12.402849177482635),
  np.float64(-0.43537242288671507),
  np.float64(-4.6397846493873285),
  nan),
 ('WES Barrier',
  np.float64(14.534481370971232),
  np.float64(0.7667226159568941),
  np.float64(-4.9408365419135425),
  nan),
 ('Basket Call',
  np.float64(4.924608118800302),
  np.float64(0.2548336608552014),
  np.float64(-52.133189663911786),
  np.float64(25.815007213063623))]

In [5]:
# ── Cell 5: Results Summary ──────────────────────────────────────────
# Assemble into a DataFrame for clear presentation
import pandas as pd

df = pd.DataFrame(results, columns=["Trade", "Price", "Delta", "Theta", "Vega"]).set_index("Trade")
df


,Price,Delta,Theta,Vega
Trade,,,,
BHP Euro Call,8.064657,0.714789,-2.022324,NaN
CBA Am Put,12.402849,-0.435372,-4.639785,NaN
WES Barrier,14.534481,0.766723,-4.940837,NaN
Basket Call,4.924608,0.254834,-52.133190,25.815007


In [7]:
# ── Cell 7: Narrative Hedge Summary ───────────────────────────────────

from IPython.display import Markdown, display

# Rebuild a small DataFrame from your `results` list
cols = ["Price","Delta","Theta","Vega"]
df_narr = (
    pd.DataFrame(results, columns=["Trade"] + cols)
      .set_index("Trade")
      .round(4)
)

# Compute hedge quantity = –Delta (i.e. short Δ to hedge)
df_narr["HedgeQty"] = (-df_narr["Delta"]).round(2)

def action(qty):
    return "buying" if qty > 0 else "selling"

# Start building the markdown text
text = "## Portfolio Hedge Summary\n\n"

# First three single‐stock/exotic legs
meta = {
    "BHP Euro Call":  ("BHP", "call"),
    "CBA Am Put":     ("CBA", "put"),
    "WES Barrier":    ("WES", "barrier call")
}

for trade, (ticker, ctype) in meta.items():
    row   = df_narr.loc[trade]
    act   = action(row.HedgeQty)
    text += (
        f"- **{trade}**  \n"
        f"  Price = **${row.Price:.4f}**, Δ = {row.Delta:.4f}, Vega = {row.Vega:.4f}, Theta = {row.Theta:.4f}  \n"
        f"  → Hedge by **{act}** {abs(row.HedgeQty):.2f} shares of {ticker} per {ctype}.\n\n"
    )

# Now the basket leg
brow   = df_narr.loc["Basket Call"]
bact   = action(brow.HedgeQty)
text += (
    f"- **Four-Stock Basket Call**  \n"
    f"  Price = **${brow.Price:.4f}**, Δ = {brow.Delta:.4f}, Vega = {brow.Vega:.4f}, Theta = {brow.Theta:.4f}  \n"
    f"  → Hedge basket by trading each underlying:  \n"
)

# Distribute across weights
weights = [0.10, 0.35, 0.15, 0.40]
tickers = ["BHP", "CSL", "WDS", "MQG"]
for w, t in zip(weights, tickers):
    sub_qty = (w * brow.HedgeQty).round(2)
    sub_act = action(sub_qty)
    text += f"    - **{sub_act}** {abs(sub_qty):.2f} shares of {t} (weight {int(w*100)}%)  \n"

display(Markdown(text))



## Portfolio Hedge Summary

- **BHP Euro Call**  
  Price = **$8.0647**, Δ = 0.7148, Vega = nan, Theta = -2.0223  
  → Hedge by **selling** 0.71 shares of BHP per call.

- **CBA Am Put**  
  Price = **$12.4028**, Δ = -0.4354, Vega = nan, Theta = -4.6398  
  → Hedge by **buying** 0.44 shares of CBA per put.

- **WES Barrier**  
  Price = **$14.5345**, Δ = 0.7667, Vega = nan, Theta = -4.9408  
  → Hedge by **selling** 0.77 shares of WES per barrier call.

- **Four-Stock Basket Call**  
  Price = **$4.9246**, Δ = 0.2548, Vega = 25.8150, Theta = -52.1332  
  → Hedge basket by trading each underlying:  
    - **selling** 0.02 shares of BHP (weight 10%)  
    - **selling** 0.09 shares of CSL (weight 35%)  
    - **selling** 0.04 shares of WDS (weight 15%)  
    - **selling** 0.10 shares of MQG (weight 40%)  
